In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as sum_, first, round
from pyspark.sql.types import DoubleType, IntegerType

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Ler o CSV, já tratando "NULL" como valor nulo
a = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .option('nullValue', 'NULL') \
              .csv('s3a://bucket-raw-upa-connect/temperatura_paciente.csv')

# Converter 'valor' para DoubleType e 'fk_paciente' para IntegerType
a = a.withColumn('valor', col('valor').cast(DoubleType())) \
     .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

a.show()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a648101c-3f88-465e-b286-03f8ba653bdc;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 313ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------

In [2]:
nulls_valor = a.select(sum_(col("valor").isNull().cast("decimal")).alias("qtd_nulls"))

# Mostra o resultado
nulls_valor.show()

+---------+
|qtd_nulls|
+---------+
|       82|
+---------+



In [3]:
# Nulls
nulls = a.filter(col('valor').isNull())

nulls.show()

# Remover linhas onde a coluna 'valor' está nula
a = a.filter(col('valor').isNotNull())

a.show()

+-----------------------+-------------------+-----+-----------+
|id_temperatura_paciente|          data_hora|valor|fk_paciente|
+-----------------------+-------------------+-----+-----------+
|                     13|2025-04-26 21:51:43| null|         69|
|                     22|2025-04-26 22:06:43| null|         65|
|                     43|2025-04-26 22:41:43| null|         72|
|                     46|2025-04-26 22:46:43| null|         26|
|                     55|2025-04-26 23:01:43| null|         83|
|                     61|2025-04-26 23:11:43| null|         72|
|                     73|2025-04-26 23:31:43| null|         73|
|                     88|2025-04-26 23:56:43| null|         53|
|                     97|2025-04-27 00:11:43| null|         41|
|                    100|2025-04-27 00:16:43| null|         59|
|                    109|2025-04-27 00:31:43| null|         60|
|                    115|2025-04-27 00:41:43| null|        113|
|                    145|2025-04-27 01:3

In [4]:
# Mostrar possíveis outliers
outliers = a.filter((col('valor') <= 34) | (col('valor') > 42))
print('Outliers encontrados:')
outliers.show()

# Remover os outliers
a = a.filter((col('valor') >= 34) & (col('valor') <= 42))

# Mostrar resultado final (sem nulos e sem outliers)
print('DataFrame final:')
a.show()

Outliers encontrados:
+-----------------------+-------------------+-----+-----------+
|id_temperatura_paciente|          data_hora|valor|fk_paciente|
+-----------------------+-------------------+-----+-----------+
|                     19|2025-04-26 22:01:43| 54.3|          6|
|                     20|2025-04-26 22:01:42| 54.5|          6|
|                     21|2025-04-26 22:01:41| 54.4|          6|
|                    187|2025-04-27 02:41:43| 14.1|         70|
|                    188|2025-04-27 02:41:42| 14.1|         70|
|                    189|2025-04-27 02:41:41| 13.9|         70|
|                    427|2025-04-27 09:21:43| 80.8|         43|
|                    428|2025-04-27 09:21:42| 80.5|         43|
|                    429|2025-04-27 09:21:41| 81.2|         43|
|                    613|2025-04-27 14:31:43| 89.9|         52|
|                    614|2025-04-27 14:31:42| 90.2|         52|
|                    615|2025-04-27 14:31:41| 89.7|         52|
|                 

In [11]:
# Agrupa por paciente, pega a primeira data_hora e calcula a média do valor
a_agrupada = a.groupBy('fk_paciente').agg(
    first('data_hora').alias('data_hora'),
    round(avg('valor'), 1).alias('media_temperatura')
)

# Ordena pelo fk_paciente (se quiser)
a_agrupada = a_agrupada.orderBy('data_hora')

# Exibe o resultado
print('DataFrame final:')
a_agrupada.show()

DataFrame final:
+-----------+-------------------+-----------------+
|fk_paciente|          data_hora|media_temperatura|
+-----------+-------------------+-----------------+
|          7|2025-04-26 21:31:43|             36.9|
|         35|2025-04-26 21:36:43|             36.8|
|         11|2025-04-26 21:41:43|             36.8|
|         92|2025-04-26 21:46:43|             36.7|
|         69|2025-04-26 21:51:42|             37.1|
|         57|2025-04-26 21:56:43|             37.0|
|         65|2025-04-26 22:06:42|             37.1|
|         28|2025-04-26 22:11:43|             36.6|
|         44|2025-04-26 22:16:43|             36.9|
|        104|2025-04-26 22:21:43|             36.1|
|         16|2025-04-26 22:26:43|             37.0|
|        127|2025-04-26 22:31:43|             37.0|
|         72|2025-04-26 22:41:42|             36.6|
|         26|2025-04-26 22:46:42|             37.0|
|          5|2025-04-26 22:51:43|             36.7|
|        111|2025-04-26 22:56:43|             3

In [9]:
a_agrupada.coalesce(1) \
    .write \
    .option('header', 'true') \
    .mode('overwrite') \
    .csv('s3a://bucket-trusted-upa-connect/temperatura_paciente_tratado.csv')

25/04/28 01:07:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/04/28 01:07:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/04/28 01:07:21 ERROR Executor: Exception in task 0.0 in stage 26.0 (TID 16)
java.lang.NoSuchMethodError: 'void org.apache.hadoop.util.SemaphoredDelegatingExecutor.<init>(com.google.common.util.concurrent.ListeningExecutorService, int, boolean)'
	at org.apache.hadoop.fs.s3a.S3AFileSystem.create(S3AFileSystem.java:772)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.s

Py4JJavaError: An error occurred while calling o108.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 16) (ip-10-100-0-35.ec2.internal executor driver): java.lang.NoSuchMethodError: 'void org.apache.hadoop.util.SemaphoredDelegatingExecutor.<init>(com.google.common.util.concurrent.ListeningExecutorService, int, boolean)'
	at org.apache.hadoop.fs.s3a.S3AFileSystem.create(S3AFileSystem.java:772)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at org.apache.spark.sql.execution.datasources.csv.CsvOutputWriter.<init>(CsvOutpu